# Cálculo de áreas de figuras geométricas con redes neuronales (¡Interactivo!)
Este notebook muestra cómo una red neuronal aprende a calcular áreas de diferentes figuras (cuadrados, rectángulos, triángulos, círculos) a partir de sus parámetros, y permite probar el modelo con entradas del usuario de manera interactiva.

Autor: Dario Coronel  
Contacto: dario.coronel@gmail.com

In [ ]:
# Instalación de librerías
# !pip install tensorflow matplotlib scikit-learn pandas ipywidgets
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

## Preparación del dataset

In [ ]:
# Dataset de ejemplo
X = np.array([
    [1,0,0,0, 4, 0, 0, 0],   # Cuadrado de lado 4 (área 16)
    [1,0,0,0, 10,0, 0, 0],   # Cuadrado de lado 10 (área 100)
    [0,1,0,0, 0, 3, 5, 0],   # Rectángulo base 3, altura 5 (área 15)
    [0,1,0,0, 0, 7, 2, 0],   # Rectángulo base 7, altura 2 (área 14)
    [0,0,1,0, 0, 8, 2, 0],   # Triángulo base 8, altura 2 (área 8)
    [0,0,1,0, 0, 6, 9, 0],   # Triángulo base 6, altura 9 (área 27)
    [0,0,0,1, 0, 0, 0, 7],   # Círculo radio 7 (área 153.938)
    [0,0,0,1, 0, 0, 0, 3],   # Círculo radio 3 (área 28.274)
    [1,0,0,0, 2, 0, 0, 0],   # Cuadrado de lado 2 (área 4)
    [0,1,0,0, 0, 10,1, 0],   # Rectángulo base 10, altura 1 (área 10)
    [0,0,1,0, 0, 3, 5, 0],   # Triángulo base 3, altura 5 (área 7.5)
    [0,0,0,1, 0, 0, 0, 5],   # Círculo radio 5 (área 78.5398)
], dtype=float)

y = np.array([
    16, 100, 15, 14, 8, 27,
    np.pi*7*7, np.pi*3*3, 4, 10, 7.5, np.pi*5*5
], dtype=float)

## Definición y entrenamiento del modelo (con EarlyStopping)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[8]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.05),
    loss='mean_squared_error'
)

history = model.fit(X, y, epochs=1000, verbose=False, callbacks=[early])

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Entrenamiento')
plt.show()

## Predicción interactiva con inputs del usuario

In [ ]:
figura = widgets.Dropdown(
    options=[('Cuadrado', 'cuadrado'), ('Rectángulo', 'rectangulo'), ('Triángulo', 'triangulo'), ('Círculo', 'circulo')],
    value='cuadrado',
    description='Figura:',
    style={'description_width': 'initial'}
)
lado = widgets.FloatSlider(value=5, min=1, max=20, step=0.1, description='Lado:', style={'description_width': 'initial'})
base = widgets.FloatSlider(value=5, min=1, max=20, step=0.1, description='Base:', style={'description_width': 'initial'})
altura = widgets.FloatSlider(value=5, min=1, max=20, step=0.1, description='Altura:', style={'description_width': 'initial'})
radio = widgets.FloatSlider(value=5, min=1, max=20, step=0.1, description='Radio:', style={'description_width': 'initial'})

inputs_box = widgets.VBox([lado])

def actualizar_inputs(*args):
    if figura.value == 'cuadrado':
        inputs_box.children = [lado]
    elif figura.value == 'rectangulo':
        inputs_box.children = [base, altura]
    elif figura.value == 'triangulo':
        inputs_box.children = [base, altura]
    else:
        inputs_box.children = [radio]

figura.observe(actualizar_inputs, names='value')

boton = widgets.Button(description='Predecir área', button_style='success')
salida = widgets.Output()

def calcular_area(figura, lado, base, altura, radio):
    if figura=='cuadrado':
        return lado**2
    elif figura=='rectangulo':
        return base*altura
    elif figura=='triangulo':
        return (base*altura)/2
    elif figura=='circulo':
        return np.pi*radio*radio

def on_button_clicked(b):
    salida.clear_output()
    # One-hot y features
    if figura.value=='cuadrado':
        x = np.array([[1,0,0,0, lado.value,0,0,0]])
        area_mat = calcular_area('cuadrado', lado.value, 0,0,0)
    elif figura.value=='rectangulo':
        x = np.array([[0,1,0,0, 0,base.value,altura.value,0]])
        area_mat = calcular_area('rectangulo', 0, base.value, altura.value, 0)
    elif figura.value=='triangulo':
        x = np.array([[0,0,1,0, 0,base.value,altura.value,0]])
        area_mat = calcular_area('triangulo', 0, base.value, altura.value, 0)
    else:
        x = np.array([[0,0,0,1, 0,0,0,radio.value]])
        area_mat = calcular_area('circulo', 0,0,0,radio.value)
    # Predicción
    area_pred = model.predict(x)[0][0]
    with salida:
        print(f'Área matemática: {area_mat:.4f}')
        print(f'Área predicha por la red: {area_pred:.4f}')
        print(f'Error absoluto: {abs(area_mat-area_pred):.4f}')

boton.on_click(on_button_clicked)

display(widgets.VBox([figura, inputs_box, boton, salida]))

---
Autor: Dario Coronel  
Contacto: dario.coronel@gmail.com